<a href="https://colab.research.google.com/github/FSDAwaesf/NLP-ffrequency-classification-and-naive-bayes-positive-and-negative/blob/main/nlpclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import re
import numpy as np
import pandas as pd

In [2]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from nltk.corpus import twitter_samples

In [4]:
positive_tweets=twitter_samples.strings('positive_tweets.json')
negative_tweets=twitter_samples.strings('negative_tweets.json')

In [5]:
len(positive_tweets)


5000

In [6]:
len(negative_tweets)

5000

In [7]:
negative_tweets[:10]

['hopeless for tmr :(',
 "Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(",
 '@Hegelbon That heart sliding into the waste basket. :(',
 '“@ketchBurning: I hate Japanese call him "bani" :( :(”\n\nMe too',
 'Dang starting next week I have "work" :(',
 "oh god, my babies' faces :( https://t.co/9fcwGvaki0",
 '@RileyMcDonough make me smile :((',
 '@f0ggstar @stuartthull work neighbour on motors. Asked why and he said hates the updates on search :( http://t.co/XvmTUikWln',
 'why?:("@tahuodyy: sialan:( https://t.co/Hv1i0xcrL2"',
 'Athabasca glacier was there in #1948 :-( #athabasca #glacier #jasper #jaspernationalpark #alberta #explorealberta #… http://t.co/dZZdqmf7Cz']

## Data Cleaning 

In [8]:
def Clean_text(text):
  text=re.sub(r'#[a-zA-Z0-9]*','',text) # delete 
  text=re.sub(r'https?:\S+','',text)
  text=re.sub(r'\d+','',text)
  text=re.sub(r'^\s+','',text)
  text=re.sub(r'\s+$','',text)
  text=re.sub(r'\W',' ',text)
  return text

In [9]:
t=Clean_text(negative_tweets[9])
t.split()

['Athabasca', 'glacier', 'was', 'there', 'in']

In [10]:
negative_tweets[9]

'Athabasca glacier was there in #1948 :-( #athabasca #glacier #jasper #jaspernationalpark #alberta #explorealberta #… http://t.co/dZZdqmf7Cz'

In [11]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
ps=PorterStemmer()
stop_words = set(stopwords.words("english"))


In [12]:
import string
punkc=string.punctuation
def preparing_wrods(tweets):
  prepared_tweet=[]
  for tweet in tweets:
    tweet=Clean_text(tweet)
    tweet=tweet.split()
    tweet=[word for word in tweet if word.lower() not in stop_words]
    tweet=[ps.stem(word) for word in tweet]
    prepared_tweet.append(tweet)
  return prepared_tweet


    


In [13]:
negative_tweets=preparing_wrods(negative_tweets)
positive_tweets=preparing_wrods(positive_tweets)

In [14]:
negative_tweets[:5]

[['hopeless', 'tmr'],
 ['everyth', 'kid', 'section', 'ikea', 'cute', 'shame', 'nearli', 'month'],
 ['hegelbon', 'heart', 'slide', 'wast', 'basket'],
 ['ketchburn', 'hate', 'japanes', 'call', 'bani'],
 ['dang', 'start', 'next', 'week', 'work']]

In [15]:
def freq_prepared(tweets,label):
  freq={}
  for tweet in tweets:
    for word in tweet:
      key=(word,label)
      if key in freq:
        freq[key]+=1
      else:
        freq[key]=1
  return freq




In [44]:
freq_neg=freq_prepared(negative_tweets,0)
freq_pos=freq_prepared(positive_tweets,1)
freq_pos

{('france_int', 1): 1,
 ('pkuchli', 1): 1,
 ('milipol_pari', 1): 1,
 ('top', 1): 32,
 ('engag', 1): 7,
 ('member', 1): 16,
 ('commun', 1): 32,
 ('week', 1): 86,
 ('lambja', 1): 1,
 ('hey', 1): 77,
 ('jame', 1): 7,
 ('odd', 1): 2,
 ('pleas', 1): 99,
 ('call', 1): 37,
 ('contact', 1): 7,
 ('centr', 1): 2,
 ('abl', 1): 8,
 ('assist', 1): 1,
 ('mani', 1): 33,
 ('thank', 1): 645,
 ('despiteoffici', 1): 1,
 ('listen', 1): 17,
 ('last', 1): 47,
 ('night', 1): 70,
 ('bleed', 1): 2,
 ('amaz', 1): 51,
 ('track', 1): 5,
 ('scotland', 1): 2,
 ('side', 1): 17,
 ('congrat', 1): 21,
 ('yeaaaah', 1): 1,
 ('yippppi', 1): 1,
 ('accnt', 1): 2,
 ('verifi', 1): 2,
 ('rqst', 1): 1,
 ('succeed', 1): 1,
 ('got', 1): 68,
 ('blue', 1): 9,
 ('tick', 1): 1,
 ('mark', 1): 3,
 ('fb', 1): 1,
 ('profil', 1): 2,
 ('day', 1): 248,
 ('bhaktisbant', 1): 17,
 ('pallaviruhail', 1): 8,
 ('one', 1): 131,
 ('irresist', 1): 2,
 ('like', 1): 233,
 ('keep', 1): 68,
 ('love', 1): 397,
 ('custom', 1): 4,
 ('wait', 1): 70,
 ('long'

## Create Features

In [31]:
def featuress(tweets,freq_pos,freq_neg):
  X=[]
  x=[]
  for tweet in tweets:
    x=[0,0,0]
    x[0]=1
    pos=0
    neg=0
    for word in tweet:
      pos+=freq_pos.get((word,1),1)
      neg+=freq_neg.get((word,0),0)
    x[1]=pos
    x[2]=neg
    X.append(x)
  
  return X



In [32]:
pos_featured=featuress(positive_tweets,freq_pos,freq_neg)
negative_featured=featuress(negative_tweets,freq_pos,freq_neg)

In [47]:
pos_featured[200]

[1, 16, 12]

In [19]:
pos_y=np.ones((5000,1))
neg_y=np.zeros((5000,1))

In [20]:
features=[]
ys=[]
for i in range(len(negative_featured)):
  features.append(pos_featured[i])
  features.append(negative_featured[i])
  ys.append(pos_y[i][0])
  ys.append(neg_y[i][0])



In [21]:
dataframe=pd.DataFrame(data=features,columns=['bias','pos_feq','neg_freq'])
dataframe['target']=ys
data=dataframe
data

,bias,pos_feq,neg_freq,target
0,1,176,73,1.0
1,1,2,5,0.0
2,1,919,505,1.0
3,1,76,130,0.0
4,1,195,159,1.0
...,...,...,...,...
9995,1,33,65,0.0
9996,1,228,143,1.0
9997,1,8,12,0.0
9998,1,109,160,1.0


In [22]:
y=data.iloc[:,-1]
x=data.iloc[:,:-1]

In [23]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy=train_test_split(x,y)

In [24]:
from sklearn.svm import SVC

In [25]:
model=SVC(gamma='auto')
model.fit(trainx,trainy)

SVC(gamma='auto')

In [26]:
y_pred=model.predict(testx)


In [27]:
from sklearn.metrics import accuracy_score,f1_score
print(accuracy_score(testy,y_pred)) 


0.64


In [34]:

sentances=["let me cry ....i have a college"] # Negative Tweet ,#pos

sentances=preparing_wrods(sentances) # clean sentances 
X_test=featuress(sentances,freq_pos,freq_neg)
if model.predict(X_test):
  print("its possitive tweet")
else:
  print("its negative tweet")

its negative tweet


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"


In [36]:
from sklearn.naive_bayes import GaussianNB
model2=GaussianNB()
model2.fit(trainx,trainy)
y_pred2=model.predict(testx)

In [37]:
print(accuracy_score(testy,y_pred2)) 


0.64


In [42]:
sentances=["this shit"] # Negative Tweet ,#pos

sentances=preparing_wrods(sentances) # clean sentances 
X_test=featuress(sentances,freq_pos,freq_neg)
if model.predict(X_test):
  print("its possitive tweet")
else:
  print("its negative tweet")

its negative tweet


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"
